In [1]:
from flask import Flask, render_template_string, request, url_for, jsonify
import os
import numpy as np
import pandas as pd
import math


In [43]:
df = pd.read_csv("/Users/keziahdutt/Downloads/Datathon 2025/df_final.csv")
df['displayed'] = 0
df['like'] = 0
df['dislike'] = 0
df['clicked'] = 0

In [44]:
all_images = []
columns = ['simple', 'nationalism', 'reinforce beliefs', 'persuasive lang', 
           'emotional_img', 'patriotic_img', 'action_img']

for _, row in df.iterrows():
    all_images.append({
        'filename': row['img_ref'], 
        'score': [row[col] for col in columns], 
        'text': row['sequence']
    })



In [45]:
#functions
##euclidean distance
def euclidean_distance(list1, dataframe, dfrow):
    return math.sqrt((list1[0] - dataframe.loc[dfrow,'simple']) ** 2 
                     + (list1[1] - dataframe.loc[dfrow,'nationalism']) ** 2
                     + (list1[2] - dataframe.loc[dfrow,'reinforce beliefs']) ** 2
                     + (list1[3] - dataframe.loc[dfrow,'persuasive lang']) ** 2
                     + (list1[4] - dataframe.loc[dfrow,'emotional_img']) ** 2
                     + (list1[5] - dataframe.loc[dfrow,'patriotic_img']) ** 2
                     + (list1[6] - dataframe.loc[dfrow,'action_img']) ** 2)
##sample values
def select_values(neutrality_score, covmatrix, number_of_posts):
    random_values = np.random.multivariate_normal(neutrality_score, covmatrix, number_of_posts)
    return random_values
##choose the closest posts
def closest_posts(random_values, posts_dataframe):
    all_closest_images = {}
    all_closest_rows ={}
    posts_dataframe['displayed'] = 0
    for i in range(len(random_values)):
        min_distance = float('inf')
        closest = None
        closest_row = None
        for row in range(len(posts_dataframe)):
            dist = euclidean_distance(random_values[i], posts_dataframe, row)
            if dist < min_distance and posts_dataframe.loc[row,'displayed']==0 and posts_dataframe.loc[row,'like']==0 and posts_dataframe.loc[row,'dislike']==0 and posts_dataframe.loc[row,'clicked']==0:
                min_distance = dist
                closest = posts_dataframe.loc[row, 'img_ref']
                closest_row = row
        all_closest_images[closest] = min_distance
        all_closest_rows[closest_row] = min_distance
        posts_dataframe.at[closest_row, 'displayed'] = 1
    return all_closest_rows


##choose posts
def select_posts(neutrality_score, covmatrix, number_of_posts, posts_dataframe):
    val = select_values(neutrality_score, covmatrix, number_of_posts)
    result = closest_posts(val, posts_dataframe)
    sorted_result = dict(sorted(result.items(), key=lambda item: item[1]))
    sorted_list = list(sorted_result.keys())
    return sorted_list

In [76]:
#Input and tuning
##neutrality_score
#A = [.5]*7
columns = ['simple', 'nationalism', 'reinforce beliefs', 'persuasive lang', 
           'emotional_img', 'patriotic_img', 'action_img']

neutrality_score = [np.mean(df[col]) for col in columns]
##covmatrix: determine variance of MVN-distribution (the larger this is, the further of your score this will be)
B = np.diag([0.1]*7)
#A = np.diag([0.001, 0.001])


In [112]:
#functions for analysis at end of game

def load_data():
    return pd.DataFrame({
        'component_1': [0.453524, 1.256381, -0.123608, 1.443164, 0.871849, 0.987512, -0.730776, -4.158047],
        'component_2': [-0.255636, 0.707914, -0.065397, -1.089624, -2.124459, 1.103933, 2.543844, -0.820575],
        'like':     [0, 1, 0, 1, 0, 1, 0, 1],
        'dislike': [1, 0, 1, 0, 1, 0, 0, 1],
        'clicked': [1, 0, 2, 0, 3, 0, 0, 4]
    })
    # return pd.read_csv('final_df.csv')

def generate_svg():
    data = load_data().sort_values(by="clicked")
    width, height = 500, 500
    min_x, max_x = -5, 2
    min_y, max_y = -3, 3

    def scale_x(x):
        return ((x - min_x) / (max_x - min_x)) * width

    def scale_y(y):
        return height - ((y - min_y) / (max_y - min_y)) * height

    svg_elements = ["<rect width='500' height='500' fill='#F5EFE0' />"]
    points = []

    for _, row in data.iterrows():
        x, y, click = scale_x(row['component_1']), scale_y(row['component_2']), row['clicked']
        points.append((x, y, click))

        # Determine color
        if row['like'] == row['dislike']:
            color = 'grey'
        elif row['like'] == 0 and row['dislike'] == 1:
            color = 'red'
        elif row['like'] == 1 and row['dislike'] == 0:
            color = 'green'

        svg_elements.append(f"<circle cx='{x}' cy='{y}' r='5' fill='{color}' stroke='black' />")

    points.sort(key=lambda p: p[2])  # Sort by click order
    clicked_points = [p for p in points if p[2] != 0]

    line_js_data = []  # Store line data for JS animation

    if clicked_points:
        start_x, start_y = width / 2, 0
        line_js_data.append(f"{{x1: {start_x}, y1: {start_y}, x2: {clicked_points[0][0]}, y2: {clicked_points[0][1]}}}")

        for i in range(len(clicked_points) - 1):
            x1, y1 = clicked_points[i][:2]
            x2, y2 = clicked_points[i + 1][:2]
            line_js_data.append(f"{{x1: {x1}, y1: {y1}, x2: {x2}, y2: {y2}}}")

    return "".join(svg_elements), line_js_data


In [118]:

app = Flask(__name__)

# create lists to store liked and disliked posts
liked_posts = set()
disliked_posts = set()
# List of available images in the static folder
folder_path = "static"
# Get a list of all file names in the folder
available_images = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]

#start page template
start_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Start Page</title>
    <link href="https://fonts.googleapis.com/css2?family=Baskervville&display=swap" rel="stylesheet">
    <style>
    body {
        display: grid;
        place-items: center;
        font-family: 'Baskervville', serif;
        background-color: #F5EFE0;
    
    }
    .start-button {
         margin-top: 30px;
            padding: 15px 30px;
            font-family: inherit;
            font-size: 30px;
            background-color: #E8DEC9;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            }
    @keyframes shrink {
        0% {transform: scale(1); }
        100% {transform: scale(0.8); }
        }
    .shrink-image {
        width: 600px;
        height: auto;
        animation: shrink 2s ease-in-out forwards;
        animation-delay: 0.3s;
    }
    </style>
</head>

<body>
    
    <img src="{{ url_for('static', filename='icons/rabbitlogo-removebg.png') }}" alt="Logo" class="shrink-image" >
    <h2 style="text-align:center;"> Welcome to Rabbit, an interactive environment <br> where you can experience World War 1 brought into the modern day. <br>
    Click start to be brought into the social media simulation. </h2>
     <form action="/home" method="get">
        <button type="submit" class="start-button">Start</button>
    </form>
</body>
</html>

"""

# Main page HTML template with clickable images and image scores
html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Flask Button Example</title>
     <style>
        body {
            background-color: #F5EFE0;
            padding-top: 10px;
        }

        .banner {
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 80px; /* Adjust height as needed */
            background-color: #E8DEC9; /* A slightly darker shade for contrast */
            display: flex;
            padding: 10px 20px;
            box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);
        }

    ]
        .banner img {
            height: 80px;
            width: auto;
            object-fit: contain;
        }
        
        .image-text-box {
            display: flex;
            flex-direction: column;
            text-overflow: ellipsis;
            justify-content: flex-end;
            text-align: center;
            height: 530px;
            width: 400px;
            background-color: #E8DEC9;
            padding: 25px;
            font-size: 24px;
            font-family: 'Baskervville', serif;
            margin-top: 100px;
            box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);
            border-radius: 15px;
            overflow: hidden; /* Ensures text doesn't overflow out of the box */
            position: relative; /* Needed to position the text correctly */
        }

        .image-container {
            display: flex;
            justify-content: center; /* Centers horizontally */
            align-items: center; /* Aligns items in the middle */
            gap: 20px; /* Adds spacing between images */
            margin-top: 200px; /* Adds some space from the top */
        
        }
        .image-container img {
            width: 400px;
            height: auto;
            display: block;
            border-radius: 15px;
        }
        .image-wrapper {
            position:relative;
        }
    .like-btn {
        position: absolute;
        bottom: 30px; /* Position the button at the bottom */
        left: 0px; /* Position the button to the left */
        background: none; /* Remove background color */
        border: none; /* Remove border */
        padding: 0; /* Remove padding */
        margin: 0; /* Remove margin (if needed) */
        cursor: pointer; /* Change cursor to pointer on hover */
    }
    .dislike-btn {
        position: absolute;
        bottom: 27px; /* Position the button at the bottom */
        left: 50px; /* Position the button to the left */
        background: none; /* Remove background color */
        border: none; /* Remove border */
        padding: 0; /* Remove padding */
        margin: 0; /* Remove margin (if needed) */
        cursor: pointer; /* Change cursor to pointer on hover */
    }
    @keyframes buttonClick {
        0% {transform: scale(1);}
        50% {transform: scale(1.3);}
        100% {transform: scale(1);}
    }
   .like-btn.clicked, .dislike-btn.clicked {
    animation: buttonClick 0.3s ease-in-out;
}
    .exit-button {
        position: absolute;
        right: 60px;
        background:none;
        border: none;
        padding: 5px;
        cursor: pointer;
    }

    .exit-button img {
        height:60px;
        width:auto;
    }
</style>
        
<script>
function handleLike(postId) {
    fetch("/like", {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify({ post_id: postId })
    })
    .then(response => response.json())
    .then(data => {
        if (data.success) {
            const likeButton = document.querySelector(`button[data-post-id='${postId}'][data-button-type='like']`);
            likeButton.classList.add("clicked");
            document.getElementById("neutrality-score").innerText = data.neutrality_score.toFixed(1);
            likeButton.addEventListener("animationend", () => {
                likeButton.classList.remove("clicked");
            }, { once: true });
        } else {
            alert(data.message);
        }
    })
    .catch(error => console.error("Error:", error));
}

function handleDislike(postId) {
    fetch("/dislike", {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify({ post_id: postId })
    })
    .then(response => response.json())
    .then(data => {
        if (data.success) {
            const dislikeButton = document.querySelector(`button[data-post-id='${postId}'][data-button-type='dislike']`);
            dislikeButton.classList.add("clicked");
            document.getElementById("neutrality-score").innerText = data.neutrality_score.toFixed(1);
            dislikeButton.addEventListener("animationend", () => {
                dislikeButton.classList.remove("clicked");
            }, { once: true });
        } else {
            alert(data.message);
        }
    })
    .catch(error => console.error("Error:", error));
}
</script>
   
</head>

<body>
        
    <div class="banner">
        <img src="{{ url_for('static', filename='icons/rabbitlogo-removebg.png') }}" alt="Logo" >
           <form action="/exit" method="get">
        <button type="submit" class="exit-button"> 
            <img src="{{ url_for('static', filename='icons/exit.png') }}" alt="Click to exit game"}}>
        </button>
        </form>
    </div>


    <div class="image-container">

        <div class="image-wrapper">
        <form method="POST" action="/button-clicked/{{ images[0].filename }}" style="display: inline;">
            <button type="submit" style="border: none; background: none; padding: 0;">
                <div class="image-text-box">
                <h3> {{ images[0].text}}</h3>
                <img src="{{ url_for('static', filename=images[0].filename) }}" style="border:3px solid black; height:400px; width:400px;" alt="Click to see {{ images[0].filename }}" >
                </div>
            </button>
        </form>
   
    
        <button class="like-btn" data-post-id="{{ images[0].filename }}" data-button-type="like" onclick="handleLike('{{ images[0].filename }}')">
            <img src="{{ url_for('static', filename='icons/happyrabbit.png') }}" alt="Like" style="width: 100px; height: auto;">
         </button>
         <button class="dislike-btn" data-post-id="{{ images[0].filename }}" data-button-type="dislike" onclick="handleDislike('{{ images[0].filename }}')">
            <img src="{{ url_for('static', filename='icons/sadrabbit.png') }}" alt="Dislike" style="width: 100px; height: auto;" >
        </button>
        </div>
        
        <div class="image-wrapper">
        <form method="POST" action="/button-clicked/{{ images[1].filename }}" style="display: inline;">
            <button type="submit" style="border: none; background: none; padding: 0;">
            <div class="image-text-box">
                <h3> {{ images[1].text}}</h3>
                <img src="{{ url_for('static', filename=images[1].filename) }}" style="border:3px solid black; height:400px; width:400px;" alt="Click to see {{ images[1].filename }}" >
            </div>
            </button>
        </form>
        <button class="like-btn" data-post-id="{{ images[1].filename }}" data-button-type="like" onclick="handleLike('{{ images[1].filename }}')">
            <img src="{{ url_for('static', filename='icons/happyrabbit.png') }}" alt="Like" style="width: 100px; height: auto;" >
        </button>
        <button class="dislike-btn" data-post-id="{{ images[1].filename }}" data-button-type="dislike" onclick="handleDislike('{{ images[1].filename }}')">
            <img src="{{ url_for('static', filename='icons/sadrabbit.png') }}" alt="Dislike" style="width: 100px; height: auto;" >
        </button>
        </div>

    </div>

    

</body>

</html>
"""

#Results Page

result_template = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Flask PCA Animation</title>
        <style>
            body {{
                display: flex;
                flex-direction: column;
                justify-content: center;
                align-items: center;
                height: 100vh;
                margin: 0;
                background-color: #F5EFE0;
                font-family: Arial, sans-serif;
            }}
            h1 {{
                margin-bottom: 20px;
                
            }}
    
            .banner {{
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 80px; /* Adjust height as needed */
            background-color: #E8DEC9; /* A slightly darker shade for contrast */
            display: flex;
            padding: 10px 20px;
            box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);
        }}

    ]
        .banner img {{
            height: 80px;
            width: auto;
            object-fit: contain;
        }}
        </style>
    </head>
    <body>
        <div class="banner">
        <img src="{{ url_for('icons/static', filename='rabbit_jump.png') }}" alt="Logo">
        </div>
        <div>
            <h1>Congratulations, you survived the war! </h1>
            <text>
             During this game you experienced several styles of reporting about the war.
            <br>
            Some of them were more neutral, while others were more propagandistic. 
            <br>
            Propagandistic messages are characterized by persuasive language, simplifying information and omitting nuance. <br>
            Instead of challenging your beliefs, they are reinforced. <br>
            Lastly, they play on your emotions and use a lot of we-vs-them speech.  <br>
            Similar to current social media, the algorithm of Rabbithole is constructed in such a way <br>
            that you see articles similar to the articles you liked before.  <br>
            Curious how far you spiraled into a rabbithole of propagandistic messages? Let's take a look! <br>
            </text>
        </div>
        <form method="POST" action="/result-clicked">
        <button type="submit">Your result</button>
        </form>
        

    </body>
    </html>
    """


@app.route("/result-clicked", methods=["POST"])
def result_clicked():
    svg_content, line_js_data = generate_svg()
    js_lines = "[" + ", ".join(line_js_data) + "]"

    result2_template = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Flask PCA Animation</title>
        <style>
            body {{
                display: flex;
                flex-direction: column;
                justify-content: center;
                align-items: center;
                height: 100vh;
                margin: 0;
                background-color: #F5EFE0;
                font-family: Arial, sans-serif;
            }}
            h1 {{
                margin-bottom: 20px;
                
            }}
           svg {{
                margin-top: 80px; /* Adjust this value to fit your needs */
            }}
            #rabbit {{
                position: absolute;
                width: 50px; /* Adjust the size of the rabbit image */
                height: auto;
                pointer-events: none;
            }}
            .banner {{
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 80px; /* Adjust height as needed */
            background-color: #E8DEC9; /* A slightly darker shade for contrast */
            display: flex;
            padding: 10px 20px;
            box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);
        }}

    ]
        .banner img {{
            height: 80px;
            width: auto;
            object-fit: contain;
        }}
        </style>
    </head>
    <body>
        <div class="banner">
        <img src="{{ url_for('icons/static', filename='rabbitlogo-removebg') }}" alt="Logo" >
        </div>
        <svg id="pca-svg" width="480" height="480">
            {svg_content}
        </svg>

        <img id="rabbit" src="icons/static/rabbit_jump.png" alt="rabbit_jump" />
    
        <script>
            const lines = {js_lines};
            const svg = document.getElementById("pca-svg");
            const rabbit = document.getElementById("rabbit");

            function animateLine(line, x1, y1, x2, y2, duration, onComplete) {{
                let startTime = null;
                
                function step(timestamp) {{
                    if (!startTime) startTime = timestamp;
                    let progress = (timestamp - startTime) / duration;
                    if (progress > 1) progress = 1;

                    line.setAttribute("x2", x1 + (x2 - x1) * progress);
                    line.setAttribute("y2", y1 + (y2 - y1) * progress);

                    // Move the rabbit image
                    const newX = x1+360 + (x2 - x1) * progress - rabbit.width / 2;
                    const newY = y1+80 + (y2 - y1) * progress - rabbit.height / 2;
                    rabbit.style.left = newX + "px";
                    rabbit.style.top = newY + "px";

                    if (progress < 1) {{
                        requestAnimationFrame(step);
                    }} else if (onComplete) {{
                        onComplete();
                    }}
                }}

                requestAnimationFrame(step);
            }}

            function drawLinesSequentially(index) {{
                if (index >= lines.length) return;

                let data = lines[index];
                let line = document.createElementNS("http://www.w3.org/2000/svg", "line");
                line.setAttribute("x1", data.x1);
                line.setAttribute("y1", data.y1);
                line.setAttribute("x2", data.x1);
                line.setAttribute("y2", data.y1);
                line.setAttribute("stroke", "black");
                line.setAttribute("stroke-width", "2");
                svg.appendChild(line);

                animateLine(line, data.x1, data.y1, data.x2, data.y2, 800, () => drawLinesSequentially(index + 1));
            }}

            drawLinesSequentially(0);
        </script>
    </body>
    </html>
    """
    return render_template_string(result2_template) 


# Template to display the clicked image
button_clicked_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Button Clicked</title>
    <style>
        body {
            background-color: #F5EFE0;
            padding-top: 5px;
        }

        .banner {
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 80px; /* Adjust height as needed */
            background-color: #E8DEC9; /* A slightly darker shade for contrast */
            display: flex;
            padding: 10px 20px;
            box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);
        }

    .home-button {
        background: none;  
        border: none;              
        font-size: 30px;         
        color: black;             
        cursor: pointer;          
        padding: none;               
        display: inline;           
        }   

        .home-button img {
            height: 80px;
            width: auto;
            object-fit: contain;             
        }
        
        
        
        .image-text-box {
            display: flex;        
            flex-direction: column; 
            text-overflow: ellipsis;
            justify-content: flex-end;
            text-align: center;
            height:530px;
            width:400px;
            background-color: #E8DEC9;
            padding: 25px;
            font-size: 24px;
            font-family: 'Baskervville', serif;
            margin-top: 100px; 
            box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);
            border-radius: 15px;
        }

        .image-container {
            display: flex;
            justify-content: center; /* Centers horizontally */
            align-items: center; /* Aligns items in the middle */
            gap: 20px; /* Adds spacing between images */
            margin-top: 200px; /* Adds some space from the top */
        
        }
        .image-container img {
            width: 400px;
            height: auto;
            display: block;
            border-radius: 15px;
        }
        .image-wrapper {
            position:relative;
        }

        
    .like-btn {
        position: absolute;
        bottom: 30px; /* Position the button at the bottom */
        left: 5px; /* Position the button to the left */
        background: none; /* Remove background color */
        border: none; /* Remove border */
        padding: 0; /* Remove padding */
        margin: 0; /* Remove margin (if needed) */
        cursor: pointer; /* Change cursor to pointer on hover */
    }
    .dislike-btn {
        position: absolute;
        bottom: 27px; /* Position the button at the bottom */
        left: 40px; /* Position the button to the left */
        background: none; /* Remove background color */
        border: none; /* Remove border */
        padding: 0; /* Remove padding */
        margin: 0; /* Remove margin (if needed) */
        cursor: pointer; /* Change cursor to pointer on hover */
    }
    @keyframes buttonClick {
        0% {transform: scale(1);}
        50% {transform: scale(1.3);}
        100% {transform: scale(1);}
    }
   .like-btn.clicked, .dislike-btn.clicked {
    animation: buttonClick 0.3s ease-in-out;
}


</style>
        
<script>
function handleLike(postId) {
    fetch("/like", {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify({ post_id: postId })
    })
    .then(response => response.json())
    .then(data => {
        if (data.success) {
            const likeButton = document.querySelector(`button[data-post-id='${postId}'][data-button-type='like']`);
            likeButton.classList.add("clicked");  // Add the "clicked" class
            document.getElementById("neutrality-score").innerText = data.neutrality_score.toFixed(1);

            // Remove the "clicked" class after the animation ends
            likeButton.addEventListener("animationend", () => {
                likeButton.classList.remove("clicked");
            }, { once: true });
        } else {
            alert(data.message);
        }
    })
    .catch(error => console.error("Error:", error));
}

function handleDislike(postId) {
    fetch("/dislike", {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify({ post_id: postId })
    })
    .then(response => response.json())
    .then(data => {
        if (data.success) {
            const dislikeButton = document.querySelector(`button[data-post-id='${postId}'][data-button-type='dislike']`);
            dislikeButton.classList.add("clicked");  // Add the "clicked" class
            document.getElementById("neutrality-score").innerText = data.neutrality_score.toFixed(1);

            // Remove the "clicked" class after the animation ends
            dislikeButton.addEventListener("animationend", () => {
                dislikeButton.classList.remove("clicked");
            }, { once: true });
        } else {
            alert(data.message);
        }
    })
    .catch(error => console.error("Error:", error));
}
</script>
   
</head>

<body>


    <div class="banner">
           <form action="/home" method="get">
        <button type="submit" class="home-button"> 
            <img src="{{ url_for('static', filename='icons/rabbitlogo-removebg.png') }}" alt="Click to go to home page"}}>
        </button>
    </form>
    </div>

    
 <div class="image-container">
    
        <div class="image-wrapper">
            <div class="image-text-box">
            <img src="{{ url_for('static', filename=image_filename) }}" style="border:3px solid black; height:400px; width:400px;" alt="Clicked Image">
            </div>

            <button class="like-btn" data-post-id="{{ image_filename }}" data-button-type="like" onclick="handleLike('{{ image_filename }}')">
                <img src="{{ url_for('static', filename='icons/happyrabbit.png') }}" alt="Like" style="width: 80px; height: auto;">
            </button>
            <button class="dislike-btn" data-post-id="{{ image_filename }}" data-button-type="dislike" onclick="handleDislike('{{ image_filename }}')">
                <img src="{{ url_for('static', filename='icons/sadrabbit.png') }}" alt="Dislike" style="width: 80px; height: auto;">
            </button> 
        </div>

        <div class="image-text-box">
            <p>{{ text }}</p>
        </div>
        
        </div>

    
</body>
</html>
"""


@app.route("/", methods=["GET"])
def start():
        return render_template_string(start_template)

@app.route("/home", methods=["GET"])
def home():
    global neutrality_score
    res = select_posts(neutrality_score,B,2,df)
    selected_images = [all_images[res[0]], all_images[res[1]]]
    return render_template_string(html_template, neutrality_score=neutrality_score, images=selected_images)
@app.route('/exit', methods=["GET"])
def exit():
    return render_template_string(result_template)

@app.route("/button-clicked/<image_filename>", methods=["POST"])
def button_clicked(image_filename):
    global neutrality_score
    
    # Recalculate the image list (just like in the home route) to get the image score
    #nr_images = len(available_images)
    #images = []
    #for idx, filename in enumerate(available_images):
        #image_score = 
        #images.append({'filename': filename, 'score': round(image_score, 2)})  # Store both filename and score
    
    # Find the image with the corresponding filename
    image = next(image for image in all_images if image['filename'] == image_filename)
    image_score = image['score']  # Get the image score
    text = image['text']
    
    return render_template_string(button_clicked_template, neutrality_score=neutrality_score, image_filename=image_filename, image_score=image_score, text=text)

def update_csv(img_ref, action):
    # Update the respective column
    if action == "like":
        df.loc[df["img_ref"] == img_ref, "like"] += 1
    elif action == "dislike":
        df.loc[df["img_ref"] == img_ref, "dislike"] += 1

    #df.to_csv(CSV_FILE, index=False)
    return True, df
@app.route("/like", methods=["POST"])

def like():
    global neutrality_score
    data = request.get_json()
    post_id = data.get('post_id')
    
    if post_id in disliked_posts:
        return jsonify({"success": False, "message": "You cannot like a post you've disliked"})
    
    liked_posts.add(post_id)
    neutrality_score = [min(1.0, score + 0.1) for score in neutrality_score]
    
    return jsonify({"success": True, "neutrality_score": neutrality_score})

@app.route('/dislike', methods=['POST'])

def dislike():
    global neutrality_score
    data = request.get_json()
    post_id = data.get('post_id')
    
    if post_id in liked_posts:
        return jsonify({"success": False, "message": "You cannot dislike a post you've liked"})
    
    disliked_posts.add(post_id)
    neutrality_score = [max(0.0, score - 0.1) for score in neutrality_score]
    
    return jsonify({"success": True, "neutrality_score": neutrality_score})

# Run the Flask app
from werkzeug.serving import run_simple

if __name__ == "__main__":
    run_simple("localhost", 5001, app)

 * Running on http://localhost:5001
Press CTRL+C to quit
127.0.0.1 - - [17/Feb/2025 21:41:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 21:41:28] "GET /static/icons/rabbitlogo-removebg.png HTTP/1.1" 304 -
127.0.0.1 - - [17/Feb/2025 21:41:29] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 21:41:29] "GET /static/icons/rabbitlogo-removebg.png HTTP/1.1" 304 -
127.0.0.1 - - [17/Feb/2025 21:41:30] "GET /static/WoslaaatogOfhmspceta.png HTTP/1.1" 304 -
127.0.0.1 - - [17/Feb/2025 21:41:30] "GET /static/icons/exit.png HTTP/1.1" 304 -
127.0.0.1 - - [17/Feb/2025 21:41:30] "GET /static/icons/happyrabbit.png HTTP/1.1" 304 -
127.0.0.1 - - [17/Feb/2025 21:41:30] "GET /static/icons/sadrabbit.png HTTP/1.1" 304 -
127.0.0.1 - - [17/Feb/2025 21:41:30] "GET /static/SdhfaofteattfTbaSabi.png HTTP/1.1" 304 -
127.0.0.1 - - [17/Feb/2025 21:41:31] "GET /exit HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 21:41:31] "GET /{%20url_for('icons/static',%20filename='rabbit_jump.png')%20} HTTP/1.1" 404 -
127.0